<a href="https://colab.research.google.com/github/ghazala-rabbani/ghazala3007/blob/main/Toxcisity_Classifier_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create Toxicity Classifier for chatbot - comments, chats, reviews, social media
# filter out - Insults,Hate speech,Threats,Harassment,Profanity
# Class 4 - Intructor Pervaiz khan
# creating model for NLP
# Text ➜ Vectorizer ➜ Numbers ➜ ML Model

# (1): create data or upload data for training

import pandas as pd

# create a Dictionay your own - dont copy and paste this data. List, Tuples, Dictionay
# Free toxicity datasets available on Kaggle to at identify toxic online behavior and reducing unintended bias

data = {
    "text": [
        "Thanks! You are a genleman",
        "This is a stupid idea",
        "I hate you",
        "Have a great day",
        "You are an idiot",
        "Thank you for your help"
    ],
    "toxic": [0, 1, 1, 0, 1, 0]  # 1 = toxic, 0 = non-toxic
}

df = pd.DataFrame(data)

In [ ]:
# (2): Vectorize text using tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words='english',
    max_features=500
)


In [ ]:
# Set your X and Y parameters with data vectorization.
X = vectorizer.fit_transform(df["text"])
y = df["toxic"]



In [ ]:
# Which model we will use - ???
# (3): The ideal model for Categorical Y values is Logistic Regression
# Logistic Regression a baseline Classification model due to its simplicity and interpretability.

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=10)
model.fit(X, y)

print("model.coef_", model.coef_)      # weights
print("model.intercept_", model.intercept_) # bias

In [ ]:
#. Predict Toxcisity
# vectors- A vectorizer is a tool that converts raw text into numerical vectors.
# Why not plain text?? Machines understand numbers in fixed-length vectors.
# It translates words, phrases, or documents into a structured, numerical format that captures semantic meaning, enabling computers to analyze patterns, calculate similarities, and perform tasks like sentiment analysis, classification, and search

def predict_toxicity(text):
    vectorized_text = vectorizer.transform([text])
    probability = model.predict_proba(vectorized_text)[0][1]
    prediction = model.predict(vectorized_text)[0]
    return prediction, round(probability * 100, 2)

# Please use your own practice data samples
samples = [
    "I really like this",
    "Thank you for the update",
    "You are stupid"
]

for text in samples:
    label, score = predict_toxicity(text)
    print(f"Text: {text}")
    print(f"Prediction: {label}, Toxicity Score: {score:.3f}\n")



Now that the model and vectorizer are set up, let's test the `predict_toxicity` function with some new input texts.

In [ ]:
# Test the predict_toxicity function with new text samples
new_samples = [
    "You are so kind and helpful!",
    "This is absolutely terrible I despise it.",
    "I hope you have a wonderful day!",
    "Go away, I don't want to see you."
]

for s in new_samples:
    label, score = predict_toxicity(s)
    print(f"Text: '{s}' | Toxic: {label} | ")

In [ ]:
# Can this work on TV, Audio etc live talk shows????
# YES - Audio → Speech-to-Text → Toxicity Classifier → Block / Mute / Review
# Speech to text AWS model
# Amazon Transcribe and Google Cloud Speech-to-Text.


In [ ]:
# Huggingface https://huggingface.io. is open LLM leader Board
# what models sit there eg. Khanamigo, Einstein Copilot(Healcare)
# Transformer is its library to access open LLMs for POC or research purpose. But Huggingface does sell models for fine-tuning etc.
!pip install transformers torch

In [ ]:
from transformers import pipeline
text_generator = pipeline("text-generation", model="gpt2")
# You can specify parameters like max_new_tokens for generation control
result = text_generator("The development of the internet has been one of the most transformative events in human history", max_new_tokens=20)
print(result)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Pretrained BERT-based toxicity model
MODEL_NAME = "unitary/toxic-bert"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

def predict_toxicity_bert(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = F.softmax(logits, dim=1)

    toxic_prob = probs[0][1].item()
    prediction = int(toxic_prob > 0.5)

    return prediction, round(toxic_prob * 100, 2)

# Test examples
samples = [
    "I hate you and your stupid ideas",
    "Thank you for your help today",
    "You are an idiot"
]

for text in samples:
    label, score = predict_toxicity_bert(text)

    print(f"Text: {text}")
    print(f"Prediction: {label}, Toxicity Score: {score:.3f}\n")